# Tutorial 9: Stokes equation

    This tutorial is under construction, but the code below is already functional.

Driver that computes the lid-driven cavity benchmark at low Reynolds numbers
when using a mixed FE Q(k)/Pdisc(k-1).

Load Gridap library

In [16]:
using Pkg
Pkg.add("Gridap")

   Resolving package versions...
  No Changes to `C:\Users\AllanStruthers\Desktop\GRIDAP\GRIDAP\Tutorials\Project.toml`
  No Changes to `C:\Users\AllanStruthers\Desktop\GRIDAP\GRIDAP\Tutorials\Manifest.toml`


In [17]:
using Gridap

Discrete model

In [18]:
n = 100
domain = (0,1,0,1)
partition = (n,n)
model = CartesianDiscreteModel(domain, partition)

CartesianDiscreteModel()

Define Dirichlet boundaries

In [5]:
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"diri1",[6,])
add_tag_from_tags!(labels,"diri0",[1,2,3,4,5,7,8])

12-element Vector{String}:
 "tag_1"
 "tag_2"
 "tag_3"
 "tag_4"
 "tag_5"
 "tag_6"
 "tag_7"
 "tag_8"
 "interior"
 "boundary"
 "diri1"
 "diri0"

Define reference FE (Q2/P1(disc) pair)

In [19]:
order = 2
reffeᵤ = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
reffeₚ = ReferenceFE(lagrangian,Float64,order-1;space=:P)

(Lagrangian(), (Float64, 1), Base.Iterators.Pairs(:space => :P))

Define test FESpaces

In [20]:
V = TestFESpace(model,reffeᵤ,labels=labels,dirichlet_tags=["diri0","diri1"],conformity=:H1)
Q = TestFESpace(model,reffeₚ,conformity=:L2,constraint=:zeromean)
Y = MultiFieldFESpace([V,Q])

MultiFieldFESpace()

Define trial FESpaces from Dirichlet values

In [21]:
u0 = VectorValue(0,0)
u1 = VectorValue(1,0)
U = TrialFESpace(V,[u0,u1])
P = TrialFESpace(Q)
X = MultiFieldFESpace([U,P])

MultiFieldFESpace()

Define triangulation and integration measure

In [22]:
degree = order
Ωₕ = Triangulation(model)
dΩ = Measure(Ωₕ,degree)

Measure()

Define bilinear and linear form

In [23]:
f = VectorValue(0.0,0.0)
a((u,p),(v,q)) = ∫( ∇(v)⊙∇(u) - (∇⋅v)*p + q*(∇⋅u) )dΩ
l((v,q)) = ∫( v⋅f )dΩ

l (generic function with 1 method)

Build affine FE operator

In [24]:
op = AffineFEOperator(a,l,X,Y)

AffineFEOperator()

Solve

In [25]:
uh, ph = solve(op)

MultiFieldFEFunction():
 num_cells: 10000
 DomainStyle: ReferenceDomain()
 Triangulation: CartesianGrid()
 Triangulation id: 15516244772141176899

Export results to vtk

In [26]:
writevtk(Ωₕ,"results",order=2,cellfields=["uh"=>uh,"ph"=>ph])

(["results.vtu"],)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*